In [1]:
# Imports
import os
import random
import base64
import logging
import pathlib
import typing
import numpy as np
import pandas as pd
import datetime as dt
import keras

2024-05-18 21:23:15.278442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 21:23:15.961216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
def calculate_line_score(line):
    player1_count = np.sum(line[:,0] == True)  # Count of Player 1's disks
    player2_count = np.sum(line[:,1] == True)  # Count of Player 2's disks

    if player2_count == 3 and player1_count == 0:
        return -3
    elif player2_count == 2 and player1_count == 0:
        return -2
    elif player2_count == 1 and player1_count == 0:
        return -1
    elif player1_count > 0 and player2_count > 0:
        return 0
    elif player2_count == 0 and player1_count == 1:
        return 1
    elif player2_count == 0 and player1_count == 2:
        return 2
    elif player2_count == 0 and player1_count == 3:
        return 3
    else:
        return 0

def generate_output_matrix(board):
    rows, cols, _ = board.shape
    output_matrix = np.zeros((69, 1), dtype=np.int8)
    k=0
    # Check horizontal lines
    for i in range(rows):
        for j in range(cols - 3):
            line = board[i, j:j+4, :]
            index = k
            k=k+1
            output_matrix[index] = calculate_line_score(line)
    # Check vertical lines
    for i in range(rows - 3):
        for j in range(cols):
            line = board[i:i+4, j, :]
            index = k
            k=k+1
            output_matrix[index] = calculate_line_score(line)
    # Check diagonal lines (\)
    for i in range(rows - 3):
        for j in range(cols - 3):
            line = board[i:i+4, j:j+4, :]
            index = k
            k=k+1
            output_matrix[index] = calculate_line_score(line.diagonal().transpose())
    # Check diagonal lines (/)
    for i in range(rows - 3):
        for j in range(3, cols):
            line = board[i:i+4, j-3:j+1, :]
            index = k
            k=k+1
            output_matrix[index] = calculate_line_score(np.fliplr(line).diagonal().transpose())
    return output_matrix

In [ ]:
data = pd.read_csv("data/csv/parsed_data_5.csv")
half_rows = len(data) // 2
data_first_half = data.iloc[half_rows:]
array_data_full = np.array([np.frombuffer(base64.b64decode(board), dtype=np.bool_).reshape((6,7,2)) for board in data_first_half["board"]])
np.save('data/npy/arrayreal_data_full.npy', array_data_full)

In [ ]:

fours_data_full = np.array([generate_output_matrix(array_data_full[i]) for i in range(len(array_data_full))])
np.save('data/npy/array_data_full.npy', fours_data_full)

In [ ]:
number_data_full = np.array([int(np.sum(array_data_full[i][:, :, 0]))%2==0 for i in range(len(array_data_full))])
np.save('data/npy/number_data_full.npy', number_data_full)

In [ ]:
target_data_full = np.array([data_first_half[f"ev{i}"].values for i in range(7)])
np.save('data/npy/target_data_full.npy', target_data_full)


In [ ]:
array_data_full = np.load(pathlib.Path("data/npy/arrayreal_data_full.npy"), mmap_mode = "r")
number_data_full = np.load(pathlib.Path("data/npy/number_data_full.npy"), mmap_mode = "r")
target_data_full = np.load(pathlib.Path("data/npy/target_data_full.npy"), mmap_mode = "r")
fours_data_full = np.load(pathlib.Path("data/npy/array_data_full.npy"),mmap_mode="r")

In [ ]:
# Load data
array_data_filename = pathlib.Path("data/npy/array_data_6.npy")
number_data_filename = pathlib.Path("data/npy/number_data_6.npy")
target_data_filename = pathlib.Path("data/npy/target_data_6.npy")

if array_data_filename.is_file() and number_data_filename.is_file() and target_data_filename.is_file():
    logging.info("Loading data from .npy files")
    array_data_full = np.load(array_data_filename, mmap_mode = "r")
    number_data_full = np.load(number_data_filename, mmap_mode = "r")
    target_data_full = np.load(target_data_filename, mmap_mode = "r")
else:
    logging.info("Loading data from .csv files and transforming them")
    data = pd.read_csv("data/csv/parsed_data_6.csv")
    half_rows = len(data) // 4
    data_first_half = data.iloc[3*half_rows:4*half_rows]
    array_data_full = np.array([np.frombuffer(base64.b64decode(board), dtype=np.bool_).reshape((6,7,2)) for board in data_first_half["board"]])
    fours_data_full = np.array([generate_output_matrix(array_data_full[i]) for i in range(len(array_data_full))])
    number_data_full = np.array([int(np.sum(array_data_full[i][:, :, 0]))%2==0 for i in range(len(array_data_full))])
    target_data_full = np.array([data_first_half[f"ev{i}"].values for i in range(7)])

In [ ]:
# Get full data sample
array_data = array_data_full
number_data = number_data_full
fours_data = fours_data_full
target_data = np.transpose(target_data_full)
print(len(target_data))
print(len(fours_data))
print(fours_data.shape)

In [ ]:
# Get smaller data sample
indices = random.sample(range(len(target_data_full[1])), int(len(target_data_full[1])/100))
array_data = []
number_data = []
target_data = []

for index in indices:
    array_data.append(array_data_full[index])
    number_data.append(number_data_full[index])
    target_data.append(target_data_full[:,index])

array_data = np.array(array_data)
number_data = np.array(number_data)
target_data = np.array(target_data)

In [3]:
# Setup variables
MODEL_NAME = "model5_2"
MODEL_PATH = pathlib.Path("models") / MODEL_NAME
LOSS = "mean_squared_error"
METRICS = ["mae"]
BATCH_SIZE = 32

In [ ]:
# Create model 1

# Define the input layers
number_input = keras.layers.Input(shape=(1,), name="number_input")
array_input = keras.layers.Input(shape=(6, 7, 2), name="array_input")

# Convolutional layers for the matrix input
conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu')(array_input)
conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu')(conv1)

# Flatten the convolutional output
flattened_conv = keras.layers.Flatten()(conv2)

# Concatenate the flattened convolutional output with the number input
concatenated_input = keras.layers.Concatenate()([number_input, flattened_conv])

# Define the dense layers
dense1 = keras.layers.Dense(128, activation='relu')(concatenated_input)
dense2 = keras.layers.Dense(64, activation='relu')(dense1)

# Output layer with 7 neurons for 7 possible moves
output = keras.layers.Dense(7, activation='sigmoid')(dense2)

# Define the model
model = keras.models.Model(inputs=[number_input, array_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss=LOSS, metrics=METRICS)

# Print model summary
model.summary()

In [ ]:
# Create model 2

# Define the input layers
number_input = keras.layers.Input(shape = (1,), name = "number_input") 
array_input = keras.layers.Input(shape = (6, 7, 2), name = "array_input")

# Flatten the array input
flattened_array = keras.layers.Flatten()(array_input)

# Concatenate the flattened array input with the number input
concatenated_input = keras.layers.Concatenate()([number_input, flattened_array])

# Dense layers for processing concatenated inputs
dense_layer_1 = keras.layers.Dense(256, activation='relu')(concatenated_input)
dense_layer_2 = keras.layers.Dense(128, activation='relu')(dense_layer_1)

# Convolutional layers for processing the array input
conv_layer_1 = keras.layers.Conv2D(64, (3, 3), activation='relu')(array_input)
conv_layer_2 = keras.layers.Conv2D(128, (3, 3), activation='relu')(conv_layer_1)
flatten_conv = keras.layers.Flatten()(conv_layer_2)

# Concatenate the output of the dense and convolutional layers
concatenated_output = keras.layers.Concatenate()([dense_layer_2, flatten_conv])

# Additional Dense layers
dense_layer_3 = keras.layers.Dense(128, activation='relu')(concatenated_input)
dense_layer_4 = keras.layers.Dense(64, activation='relu')(dense_layer_3)
dense_layer_5 = keras.layers.Dense(32, activation='relu')(dense_layer_4)
dense_layer_6 = keras.layers.Dense(16, activation='relu')(dense_layer_5)

# Output layer
output = keras.layers.Dense(7, activation='sigmoid',name="output")(dense_layer_6)  # 7 output neurons, one for each column

# Define the model
model = keras.models.Model(inputs=[number_input, array_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss=LOSS, metrics=METRICS)

# Print model summary
model.summary()

In [ ]:
# Create model 3

# Define the input layers
number_input = keras.layers.Input(shape=(1,), name="number_input")
array_input = keras.layers.Input(shape=(6, 7, 2), name="array_input")

# Convolutional layers for the matrix input
conv1 = keras.layers.Conv2D(1024, (3, 3), activation='relu')(array_input)
conv2 = keras.layers.Conv2D(2048, (3, 3), activation='relu')(conv1)

# Flatten the convolutional output
flattened_conv = keras.layers.Flatten()(conv2)

# Concatenate the flattened convolutional output with the number input
concatenated_input = keras.layers.Concatenate()([number_input, flattened_conv])

# Define the dense layers
dense1 = keras.layers.Dense(2048, activation='relu')(concatenated_input)
dense2 = keras.layers.Dense(1024, activation='relu')(dense1)

# Output layer with 7 neurons for 7 possible moves
output = keras.layers.Dense(7, activation='sigmoid')(dense2)

# Define the model
model = keras.models.Model(inputs=[number_input, array_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss=LOSS, metrics=METRICS)

# Print model summary
model.summary()

In [ ]:
# Create model 4

# Define the input layers
number_input = keras.layers.Input(shape = (1,), name = "number_input") 
array_input = keras.layers.Input(shape = (6, 7, 2), name = "array_input")

# Flatten the array input
flattened_array = keras.layers.Flatten()(array_input)

# Concatenate the flattened array input with the number input
concatenated_input = keras.layers.Concatenate()([number_input, flattened_array])

# Dense layers for processing concatenated inputs
dense_layer_1 = keras.layers.Dense(2048, activation='relu')(concatenated_input)
dense_layer_2 = keras.layers.Dense(1024, activation='relu')(dense_layer_1)

# Convolutional layers for processing the array input
conv_layer_1 = keras.layers.Conv2D(1024, (3, 3), activation='relu')(array_input)
conv_layer_2 = keras.layers.Conv2D(2048, (3, 3), activation='relu')(conv_layer_1)
flatten_conv = keras.layers.Flatten()(conv_layer_2)

# Concatenate the output of the dense and convolutional layers
concatenated_output = keras.layers.Concatenate()([dense_layer_2, flatten_conv])

# Additional Dense layers
dense_layer_3 = keras.layers.Dense(1024, activation='relu')(concatenated_input)
dense_layer_4 = keras.layers.Dense(512, activation='relu')(dense_layer_3)
dense_layer_5 = keras.layers.Dense(256, activation='relu')(dense_layer_4)
dense_layer_6 = keras.layers.Dense(128, activation='relu')(dense_layer_5)

# Output layer
output = keras.layers.Dense(7, activation='sigmoid',name="output")(dense_layer_6)  # 7 output neurons, one for each column

# Define the model
model = keras.models.Model(inputs=[number_input, array_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss=LOSS, metrics=METRICS)

# Print model summary
model.summary()

In [ ]:
# Create model 5

# Define the input layers
number_input = keras.layers.Input(shape = (1,), name = "number_input") 
array_input = keras.layers.Input(shape = (6, 7, 2), name = "array_input")
fours_input = keras.layers.Input(shape = (69,1), name="fours_input")

# Flatten the array input
flattened_array = keras.layers.Flatten()(array_input)
flattened_fours = keras.layers.Flatten()(fours_input)

dense_additional = keras.layers.Dense(128, activation='relu')(fours_input)
flattened_additional = keras.layers.Flatten()(dense_additional)

# Concatenate the flattened array input with the number input
concatenated_input = keras.layers.Concatenate()([number_input, flattened_array, flattened_additional])

# Dense layers for processing concatenated inputs
dense_layer_1 = keras.layers.Dense(2048, activation='relu')(concatenated_input)
dense_layer_2 = keras.layers.Dense(1024, activation='relu')(dense_layer_1)

# Convolutional layers for processing the array input
conv_layer_1 = keras.layers.Conv2D(1024, (3, 3), activation='relu')(array_input)
conv_layer_2 = keras.layers.Conv2D(2048, (3, 3), activation='relu')(conv_layer_1)
flatten_conv = keras.layers.Flatten()(conv_layer_2)

# Concatenate the output of the dense and convolutional layers
concatenated_output = keras.layers.Concatenate()([dense_layer_2, flatten_conv])

# Additional Dense layers
dense_layer_3 = keras.layers.Dense(1024, activation='relu')(concatenated_input)
dense_layer_4 = keras.layers.Dense(512, activation='relu')(dense_layer_3)
dense_layer_5 = keras.layers.Dense(256, activation='relu')(dense_layer_4)
dense_layer_6 = keras.layers.Dense(128, activation='relu')(dense_layer_5)

# Output layer
output = keras.layers.Dense(7, activation='linear',name="output")(dense_layer_6)  # 7 output neurons, one for each column

# Define the model
model = keras.models.Model(inputs=[number_input, array_input,fours_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss=LOSS, metrics=METRICS)

# Print model summary
model.summary()

In [ ]:
# Fit model
model.fit([number_data, array_data,fours_data], target_data, epochs = 3, batch_size = BATCH_SIZE, validation_split = 0.2)

In [ ]:
# Test model
test_loss, test_mertics = model.evaluate([number_data, array_data,fours_data], target_data, batch_size = BATCH_SIZE)
print(f"Test loss: {test_loss}, Test metrics: {test_mertics}")

In [ ]:
# Save model
model.save(MODEL_PATH / f"{dt.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")

In [ ]:


# Train model
while True:
    model_name = sorted([f for f in os.listdir(MODEL_PATH) if os.path.isfile(os.path.join(MODEL_PATH, f)) and f.endswith(".keras")])[-1]
    model: typing.Any = keras.models.load_model(MODEL_PATH / model_name)
    model.compile(optimizer = "adam", loss = LOSS, metrics = METRICS)
    model.fit([number_data, array_data,fours_data], target_data, epochs = 1, batch_size = BATCH_SIZE, validation_split = 0.2)
    model.save(MODEL_PATH / f"{dt.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")